In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action='ignore')

In [23]:
train_df=pd.read_csv('./data/titanic/train.csv')

train_df['Family']=train_df['SibSp']+train_df['Parch']

train_df['Title']=train_df['Name'].str.extract('([A-Za-z]+)\.')

title_mapping_dict={'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Dr': 3,
                    'Rev': 3, 'Major': 3, 'Col': 3, 'Mlle': 3, 'Ms':3, 'Don': 3, 'Dona':3,
                    'Capt':3, 'Mme':3, 'Lady':3, 'Sir':3, 'Jonkheer':3, 'Countess':3}
train_df['Title']=train_df['Title'].map(title_mapping_dict)

train_df.drop('Name', inplace=True, axis=1)
train_df.drop('Ticket', inplace=True, axis=1)
train_df.drop('Cabin', inplace=True, axis=1)
train_df.drop('PassengerId', inplace=True, axis=1)

sex_mapping_dict={'male': 0, 'female': 1}
train_df['Sex']=train_df['Sex'].map(sex_mapping_dict)

train_df['Embarked'].fillna('S', inplace=True)

embarked_mapping_dict={'S': 0, 'Q': 1, 'C':2}
train_df['Embarked']=train_df['Embarked'].map(embarked_mapping_dict)
train_df.head()

age_mean=train_df.groupby('Title')['Age'].mean()
a=train_df[train_df['Title']==0]['Age'].fillna(age_mean[0])
b=train_df[train_df['Title']==1]['Age'].fillna(age_mean[1])
c=train_df[train_df['Title']==2]['Age'].fillna(age_mean[2])
d=train_df[train_df['Title']==3]['Age'].fillna(age_mean[3])
result_series=pd.concat([a,b,c,d])
result_series.sort_index()
train_df['Age']=result_series.sort_index()

train_df.loc[train_df['Age']<=20,'Age']=0
train_df.loc[(train_df['Age']>20) & (train_df['Age']<=40),'Age']=1
train_df.loc[(train_df['Age']>40) & (train_df['Age']<=60),'Age']=2
train_df.loc[train_df['Age']>60,'Age']=3

train_df.loc[train_df['Fare']<=10,'Fare']=0
train_df.loc[(train_df['Fare']>10) & (train_df['Fare']<=100),'Fare']=1
train_df.loc[train_df['Fare']>100,'Fare']=2

columns=['Pclass', 'Sex', 'Age', 'Embarked', 'Family', 'Title']

train_df[columns].shape
train_num=int(train_df[columns].shape[0]*.8)
train_num  # 712

# train, test data set
train_x_data=train_df[columns][:train_num].values
test_x_data=train_df[columns][train_num:].values

train_y_data=train_df['Survived'][:train_num].values.reshape([-1,1])
test_y_data=train_df['Survived'][train_num:].values.reshape([-1,1])

train_x_data.shape

(712, 6)

In [24]:
tf.reset_default_graph()

# placeholder
X=tf.placeholder(shape=[None,6],dtype=tf.float32)
Y=tf.placeholder(shape=[None,1],dtype=tf.float32)

# Weight & bias ( Deep & Wide )
W1=tf.get_variable('weight1',
                   shape=[6,18],
                   initializer=tf.contrib.layers.xavier_initializer())
b1=tf.Variable(tf.random_normal([18]), name='bias1')

layer1=tf.nn.relu(tf.matmul(X,W1)+b1)

W2=tf.get_variable('weight2',
                   shape=[18,18],
                   initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.random_normal([18]), name='bias2')

layer2=tf.nn.relu(tf.matmul(layer1,W2)+b2)

W3=tf.get_variable('weight3',
                   shape=[18,1],
                   initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.random_normal([1]), name='bias3')

# Hypothesis
logit=tf.matmul(layer2,W3)+b3
H=tf.nn.sigmoid(logit)

# Cost
cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit,
                                                            labels=Y))

# train
train=tf.train.AdamOptimizer(learning_rate=.01).minimize(cost)

# session & initializer
sess=tf.Session()
sess.run(tf.global_variables_initializer())

cnt=0
# learning
for step in range(10000):
    _, cost_val=sess.run([train, cost],
                         feed_dict={X:train_x_data,
                                    Y:train_y_data})
    if step%1000==0:
        cnt+=1
        print(f'cost_value{cnt}: {cost_val}')
        
predict=tf.cast(H>.6, dtype=tf.float32)  # 사용자 임의 지정 // 통상적으로 .5
                                         # 확률로 나온 값은 0 | 1로 변환
correct=tf.equal(predict,Y)
accuracy=tf.reduce_mean(tf.cast(correct, dtype=tf.float32)) # correct가 boolean형이기에 실수로 변환

print(f'\naccuracy rate: {sess.run(accuracy, feed_dict={X:test_x_data, Y:test_y_data})}')

cost_value1: 0.6847228407859802
cost_value2: 0.3543005585670471
cost_value3: 0.3420151174068451
cost_value4: 0.3398732841014862
cost_value5: 0.33759450912475586
cost_value6: 0.3376533091068268
cost_value7: 0.335522323846817
cost_value8: 0.3370189070701599
cost_value9: 0.3351181447505951
cost_value10: 0.336544007062912

accuracy rate: 0.8491619825363159


In [28]:
test_df=pd.read_csv('./data/titanic/test.csv')
display(test_df.head())

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [29]:
test_df['Family']=test_df['SibSp']+test_df['Parch']

test_df['Title']=test_df['Name'].str.extract('([A-Za-z]+)\.')

test_df['Title']=test_df['Title'].map(title_mapping_dict)

test_df.drop('Name', inplace=True, axis=1)
test_df.drop('Ticket', inplace=True, axis=1)
test_df.drop('Cabin', inplace=True, axis=1)

test_df['Sex']=test_df['Sex'].map(sex_mapping_dict)

test_df['Embarked'].fillna('S', inplace=True)

test_df['Embarked']=test_df['Embarked'].map(embarked_mapping_dict)
test_df.head()

age_mean=test_df.groupby('Title')['Age'].mean()
a=test_df[test_df['Title']==0]['Age'].fillna(age_mean[0])
b=test_df[test_df['Title']==1]['Age'].fillna(age_mean[1])
c=test_df[test_df['Title']==2]['Age'].fillna(age_mean[2])
d=test_df[test_df['Title']==3]['Age'].fillna(age_mean[3])
result_series=pd.concat([a,b,c,d])
result_series.sort_index()
test_df['Age']=result_series.sort_index()

test_df.loc[test_df['Age']<=20,'Age']=0
test_df.loc[(test_df['Age']>20) & (test_df['Age']<=40),'Age']=1
test_df.loc[(test_df['Age']>40) & (test_df['Age']<=60),'Age']=2
test_df.loc[test_df['Age']>60,'Age']=3

test_df=test_df.fillna(0)
test_df.loc[test_df['Fare']<=10,'Fare']=0
test_df.loc[(train_df['Fare']>10) & (train_df['Fare']<=100),'Fare']=1
test_df.loc[test_df['Fare']>100,'Fare']=2

test_df.info()

x_data=test_df[columns].values
result=tf.cast(sess.run(H>.5, feed_dict={X:x_data}), dtype=tf.int32)
rr=sess.run(result)
rr.reshape([22,-1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked       418 non-null int64
Family         418 non-null int64
Title          418 non-null int64
dtypes: float64(2), int64(8)
memory usage: 32.8 KB


array([[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1],
       [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0,

In [30]:
result_df = pd.DataFrame(data=sess.run(result),
                         index=test_df['PassengerId'],
                         columns=['Survived'])
result_df

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [31]:
result_df.to_csv('./data/titanic/result_df7.csv',
                 sep=',',
                 index = True) # do not write index